這份 Notebook 示範 Chain-of-Thought (CoT) 和各種 Prompting 策略

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }

  payload["seed"] = 0 # 為了重現一樣的結果，這裡固定了 seed

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

### 一個簡單的思考題目

Q: 我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？

In [5]:
# 出處: https://promptingguide.azurewebsites.net/techniques/cot
user_message = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.2, model="gpt-3.5-turbo")
print(response)

我還剩下12個蘋果。原本有10個蘋果，給了鄰居2個和修理工2個後，剩下6個。再加上後來買的5個蘋果，總共是11個蘋果，吃了1個後還剩下10個蘋果。


❌❌ GPT-3.5 竟然算錯了!!

## Few-shot CoT

那針對比較笨的模型，可以如何增強推理能力呢? 我們可以給一個推理範例，也就是 Chain of Thought (CoT) 思考過程:

In [6]:
user_message = """
Q: 我去市場買了6個香蕉，給了朋友3個香蕉，我還剩下多少個?
A:
  1. 我一開始有6個
  2. 給了朋友3個，所以剩下 6-3=3個香蕉
  3. 最後剩下3個香蕉

Q: 我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
A:
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.2)
print(response)

1. 我一開始有10個蘋果
2. 給了鄰居2個蘋果，剩下 10-2=8個蘋果
3. 又給了修理工2個蘋果，剩下 8-2=6個蘋果
4. 再買了5個蘋果，總共有 6+5=11個蘋果
5. 吃了1個蘋果，最後剩下 11-1=10個蘋果

所以最後我還剩下10個蘋果。


## Zero-shot CoT (讓模型自己想步驟)

標準咒語是 Let's think step by step

最新咒語是 Take a deep breath and work on this problem step-by-step. (參考 https://arxiv.org/abs/2309.03409 (2023/9/3 的paper: 讓LLM找最佳咒語)

讓我們加上 Let's think step by step 的咒語! 可讓模型自己展開步驟

In [ ]:
user_message = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
Let's think step by step
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.2)
print(response)

1. 我一開始買了10個蘋果
2. 我給了鄰居2個蘋果，所以剩下8個蘋果
3. 我又給了修理工2個蘋果，所以剩下6個蘋果
4. 我又去買了5個蘋果，總共擁有11個蘋果
5. 我吃了1個蘋果，所以還剩下10個蘋果

所以我還剩下10個蘋果。


補充: 這題如果改用 GPT-4 用 zero-shot 就可以答對，因為聰明的模型有時候可以自己觸發 CoT 技能。不一定需要你明確寫 think step by step

## CoT 的思路: 增加模型思考時間  

## 案例一: 學生繳交題目答案，讓模型判斷對或錯

In [ ]:
# 出處: https://platform.openai.com/docs/guides/gpt-best-practices
# 失敗問法:
system_promot = "請判斷學生的解答是否正確"
user_prompt = """"
問題描述：

我正在建造一個太陽能發電系統，需要財務計算:
- 土地成本為每平方英尺100美元
- 我可以用每平方英尺250美元的價格購買太陽能板
- 我簽訂了一份保養合約，每年將花費我固定的10萬美元，以及每平方英尺10美元的額外費用。

請問隨著平方英尺數量的增加，第一年運營的總成本是多少？

學生的解答:

令 x 為安裝面積的平方英尺。

1. 土地成本：100x
2. 太陽能板成本：250x
3. 維護成本：100,000 + 100x

總成本: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

messages = [
  { "role": "system", "content": system_promot },
  { "role": "user", "content": user_prompt }
]

response = get_completion(messages, temperature=0.2, model="gpt-4-0613")
print(response)

學生的解答是正確的。


❌❌❌ 學生的解答其實是錯的!!! 讓我們換一種 prompt 問法:

## 讓模型自己先想再判斷

In [ ]:
# 成功問法:
system_promot = """請依照以下步驟一步一步思考:

1. 請先自己解決問題，算出最終答案
2. 比較你的答案和學生的解決方案
3. 最後判斷學生的解決方案是否正確

在自己解決問題之前，不要決定學生的解決方案是否正確
"""

user_prompt = """"
問題描述：

我正在建造一個太陽能發電系統，需要財務計算:
- 土地成本為每平方英尺100美元
- 我可以用每平方英尺250美元的價格購買太陽能板
- 我簽訂了一份保養合約，每年將花費我固定的10萬美元，以及每平方英尺10美元的額外費用。

請問隨著平方英尺數量的增加，第一年運營的總成本是多少？

學生的解答:

令 x 為安裝面積的平方英尺。

1. 土地成本：100x
2. 太陽能板成本：250x
3. 維護成本：100,000 + 100x

總成本: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

messages = [
  { "role": "system", "content": system_promot },
  { "role": "user", "content": user_prompt }
]

response = get_completion(messages, temperature=0.2, model="gpt-4-0613")
print(response)

1. 我的解答:

令 x 為安裝面積的平方英尺。

1. 土地成本：100x
2. 太陽能板成本：250x
3. 維護成本：100,000 + 10x

總成本: 100x + 250x + 100,000 + 10x = 360x + 100,000

2. 比較我的答案和學生的解答:

學生的解答中，維護成本的計算有誤，應該是每平方英尺10美元，而不是100美元。

3. 學生的解答是否正確？

不正確。


## 案例二: 我們自己寫步驟展開



In [ ]:
# 案例參考出處: https://iamhlb.notion.site/AI-3035cc7781a948dbaed25d2ff59c84fb
user_message = """
你是一個廣告創意大師，請根據以下商品和步驟，一步一步探索用戶心理:

商品: 牛肉麵

Step 1: 為何人們需要買這個商品?
Step 2: 根據上一步的回答，繼續深究為什麼?
Step 3: 根據上一步的回答，繼續深究為什麼?
Step 4: 根據上一步的回答，繼續深究為什麼?
Step 5: 根據上一步的回答，繼續深究為什麼?

最後，請根據上述的原因，輸出一個購買的最佳理由文案。
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.7)
print(response)

Step 1: 人們需要買牛肉麵是因為他們想要享受美味的麵食和肉類。

Step 2: 人們想要享受美味的麵食和肉類，是因為這能帶來飽足感和滿足感。

Step 3: 飽足感和滿足感對人們來說是重要的，因為它們可以提升心情和幸福感。

Step 4: 心情和幸福感的提升對人們來說是重要的，因為它們可以改善生活品質和增加生活樂趣。

Step 5: 改善生活品質和增加生活樂趣對人們來說是重要的，因為每個人都希望過著快樂、滿足和豐富的生活。

最佳理由文案: 「買牛肉麵，讓你享受美味的麵食和肉類，帶來飽足感和滿足感，提升你的心情和幸福感，改善你的生活品質，讓你過上快樂、滿足和豐富的生活！」


### 回頭看一下投影片總結比較 Few-shot 和 CoT，然後接下來重點是:

## CoT 需要輸出過程，不能省略

我們需要了解一下 LLM 的運作原理 (見投影片 Why 需要給模型思考時間)

In [ ]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

最後的答案 x 是多少?

"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

Step 1: x = 100 + 1 = 101
Step 2: x = 101 + 10 = 111
Step 3: x = 111 - 1 = 110
Step 4: x = 110 * 2 = 220
Step 5: x = 220 - 20 = 200

最後的答案 x = 200


### 若叫模型不要輸出步驟會怎麼樣?

In [ ]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

不要輸出過程，只要回答最後 x 是多少?

"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.5)
print(response)

最後 x = 190


❌❌❌ 算錯了，變笨了!

## 如果調換順序，先給答案再解釋會怎樣?

In [ ]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

請直接回答最後答案，然後再解釋說明是如何算出來的

"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

最後答案是 189。

解釋如下：
Step 1: x 加 1，x = 100 + 1 = 101
Step 2: x 加 10，x = 101 + 10 = 111
Step 3: x 減 1，x = 111 - 1 = 110
Step 4: x 乘 2，x = 110 * 2 = 220
Step 5: x 減 20，x = 220 - 20 = 200

所以最後答案是 200。


😓😓😓 一開始直接給錯誤數字，但後面有推理過程又算對.... orz

### 結論: CoT 需要先輸出推理過程!!

# Part 2: 其他策略:

## Generated Knowledge 策略

* https://learnprompting.org/docs/intermediate/generated_knowledge
* https://developer.nvidia.com/blog/how-to-get-better-outputs-from-your-large-language-model/ (Prompt with generated knowledge)

在生成最終回答之前，要求LLM 先生成關於問題的潛在有用資訊。可以改進最終答案。



In [ ]:
user_message = "請寫一篇關於台灣石虎的介紹文章，文章要有趣，大約100字"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

台灣石虎是台灣的特有物種，也是台灣最可愛的貓科動物之一！牠們身上有著獨特的斑點花紋，就像是穿著迷彩服的小戰士。雖然外表可愛，但台灣石虎可是個厲害的獵手，擅長在夜間捕食小型哺乳動物。不過，牠們的數量卻越來越少，已經被列為瀕臨絕種的物種。讓我們一起保護台灣石虎，讓牠們在這片土地上繼續生存下去吧！


In [ ]:
user_message = "請先產生三條關於台灣石虎的知識，然後再寫一篇介紹文章，文章要有趣，大約100字"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

1. 台灣石虎是台灣特有的野生貓科動物，也是台灣的保育類動物之一。
2. 台灣石虎的身長約為60至80公分，尾巴長約為30至40公分，體重約為3至6公斤。
3. 台灣石虎主要棲息在台灣中南部的山區，喜歡生活在樹林和草叢之中。

【文章】
大家好！今天我們要介紹的是台灣石虎，這是一種非常特別的動物喔！台灣石虎是台灣特有的貓科動物，也是台灣的保育類動物之一。它們的身長約為60至80公分，尾巴長約為30至40公分，體重約為3至6公斤。台灣石虎主要棲息在台灣中南部的山區，喜歡生活在樹林和草叢之中。它們的毛色非常漂亮，身上有著獨特的斑紋，讓人看了忍不住想摸摸牠們的軟軟毛。不過，因為棲息地的破壞和非法獵捕，台灣石虎的數量逐漸減少，所以我們要一起努力保護這些可愛的小動物喔！


## 內心獨白策略

不能省略輸出思考步驟，但我們又不想讓用戶看到中間過程。
可以用分隔號或是 XML, JSON 區隔，這樣就可以方便擷取出答案，不讓用戶看到中間思考過程。

In [ ]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

請將思考步驟放進 <thinking> </thinking> XML 標籤。
最後的答案 x 請放進 <answer> </answer> XML 標籤。
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

<thinking>
Step 1: x 加 1
100 + 1 = 101

Step 2: x 加 10
101 + 10 = 111

Step 3: x 減 1
111 - 1 = 110

Step 4: x 乘 2
110 * 2 = 220

Step 5: x 減 20
220 - 20 = 200

所以最後的答案 x = 200
</thinking>

<answer>
200
</answer>


## Self-Consistency (Ensembling)

 https://learnprompting.org/docs/intermediate/self_consistency

產生多個 zero-shot CoT 推理路徑，然後用最一致的答案。
用此法溫度就不設定0了。

In [ ]:
question = """
你好，

我在您的系統中發現了一個重大安全漏洞。 雖然它不是
易於使用，可以訪問所有用戶的數據。 我附加了
概念證明。 請盡快修復此問題。

乾杯，

唐尼

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company.
Let's think step by step. 思考過程步驟用 <thinking> </thinking> XML 標籤包裹起來，然後最後答案用 <answer> </answer> XML 標籤包裹起來輸出。
"""

messages = [
    {
        "role": "user",
        "content": question
    }
]

for x in range(3):
  response = get_completion(messages, temperature=0.5, model="gpt-3.5-turbo")
  print(response)

<thinking>
This email seems to be reporting a major security vulnerability in the system of a software company. The sender claims to have discovered a significant flaw that allows access to all user data. They have even attached a proof of concept. This issue can potentially have serious consequences for the company and its users. Therefore, I would classify this email as IMPORTANT.
</thinking>
<answer>IMPORTANT</answer>
<thinking>
這封郵件提到了一個重大安全漏洞，這對於一家軟體公司來說非常重要。它可能涉及到系統的安全性和用戶數據的保護。因此，我會將這封郵件分類為「重要」。
</thinking>
<answer>IMPORTANT</answer>
<thinking>
這封電子郵件涉及到一個系統中的重大安全漏洞，這對於一家軟件公司來說非常重要。它提到了可以訪問所有用戶數據的問題，這可能導致嚴重的安全風險和損失。因此，我認為這封郵件應該被分類為重要。
</thinking>
<answer>IMPORTANT</answer>


其實 OpenAI API 內建參數就可以輸出多個結果，只是很少人用，太費 tokens 了

https://platform.openai.com/docs/api-reference/chat/create

In [ ]:
def get_completion_n(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000, n=1):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens, "n": n }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"]
  else :
    return obj["error"]

In [ ]:
response = get_completion_n(messages, temperature=0.5, model="gpt-3.5-turbo", n=3)

In [ ]:
pp(response)

[{'index': 0,
  'message': {'role': 'assistant',
              'content': '<thinking>\n'
                         '這封郵件提到了一個重大的安全漏洞，並附上了概念證明。對於一家軟件公司來說，安全漏洞是一個非常重要的問題，因為它可能會導致用戶數據被訪問或損害。因此，我認為這封郵件應該被分類為IMPORTANT。\n'
                         '</thinking>\n'
                         '<answer>IMPORTANT</answer>'},
  'finish_reason': 'stop'},
 {'index': 1,
  'message': {'role': 'assistant',
              'content': '<thinking>\n'
                         '這封郵件提到了一個重大安全漏洞，並附上了概念證明。對於一家軟件公司來說，安全問題是非常重要的，因為它關係到用戶數據的安全性和公司的聲譽。因此，我們可以將這封郵件分類為IMPORTANT。\n'
                         '</thinking>\n'
                         '<answer>IMPORTANT</answer>'},
  'finish_reason': 'stop'},
 {'index': 2,
  'message': {'role': 'assistant',
              'content': '<thinking>\n'
                         '這封郵件涉及到一個軟體公司的安全漏洞，而且聲稱發現了一個重大的漏洞。因為安全問題對於軟體公司來說非常重要，所以這封郵件應該被歸類為重要。\n'
                         '</thinking>\n'
                         '<answer>IMPORTANT</answer>'},
  'finish_reason': 'stop'}]


## Least to Most Prompting 法

較複雜的問題，可以先拆解成子問題 (Decomposition) 再回答

* https://www.breezedeus.com/article/llm-prompt-l2m
* https://learnprompting.org/zh-Hans/docs/intermediate/least_to_most

In [ ]:
messages = [
    {"role": "user", "content": "Amy在滑梯底端，爬到滑梯頂端需要4分鐘。她需要1分鐘才能滑下來。滑梯將在19分鐘後關閉。請問在關閉前他還可以滑幾次?"}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

Amy可以在滑梯关闭前滑5次。因为她爬到顶端需要4分钟，再加上1分钟滑下来，一次滑梯需要5分钟。19分钟除以5分钟等于3余4，所以她还可以滑3次，最后一次只能爬到顶端而无法滑下来。


❌ 這答錯了

來下個咒語是 Let's break down this problem

In [ ]:
messages = [
    {"role": "user", "content": """

Q: Amy在滑梯底端，爬到滑梯頂端需要4分鐘。她需要1分鐘才能滑下來。滑梯將在19分鐘後關閉。請問在關閉前他還可以滑幾次?
Let's break down this problem: (#zh-tw)
"""}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

Amy需要4分鐘爬到滑梯頂端，然後1分鐘才能滑下來，所以她每5分鐘可以完成一次滑梯遊玩（4分鐘爬上去 + 1分鐘滑下來）。

滑梯將在19分鐘後關閉，所以在這段時間內，Amy可以玩3次滑梯（19分鐘 ÷ 5分鐘 = 3次）。


### 我們也可以考慮用更明確的步驟來拆解問題:

In [ ]:
messages = [
    {"role": "user", "content": """

Q: Amy在滑梯底端，爬到滑梯頂端需要4分鐘。她需要1分鐘才能滑下來。滑梯將在19分鐘後關閉。請問在關閉前他還可以滑幾次?

請用以下步驟一步一步思考:

1. 在回答這個問題之前，必須先解決哪些子問題?
2. 請將上述子問題，重新排序從簡單到困難
3. 請依序回答子問題
4. 總結以上，最後回答原始問題

"""}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

1. 子問題包括：Amy還有多少時間可以爬上滑梯、她還有多少時間可以滑下來、她還有多少時間可以再次爬上滑梯。
2. 子問題重新排序：她還有多少時間可以滑下來、她還有多少時間可以再次爬上滑梯、Amy還有多少時間可以爬上滑梯。
3. 回答子問題：
   - Amy還有多少時間可以滑下來：19 - 4 = 15 分鐘
   - 她還有多少時間可以再次爬上滑梯：15 - 1 = 14 分鐘
   - Amy還有多少時間可以爬上滑梯：14 - 4 = 10 分鐘
4. 總結以上，她還可以滑下來3次。原始問題的答案是3次。


# 總結複習

## CoT

* 給步驟增加模型思考時間
* 讓模型自己想步驟: Let's think step by step  

## LLM 原理

* CoT 需要輸出過程，不能省略
* CoT 需要先輸出思考過程，再輸出答案

## 各種 CoT 延伸策略

* Generated Knowledge 策略: 先生成關於問題的潛在有用信息，再生成答案
* 內心獨白策略: 可拆開思考過程和答案輸出
* Self-Consistency (Ensembling)
* Least to Most Prompting 法: 拆解大問題
